<a href="https://colab.research.google.com/github/RepoAnalysis/RepoSim/blob/main/notebooks/BiEncoder/UniXCoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install `inspect4py` using Python 3.9

In [ ]:
# Install python 3.9 on colab
!sudo apt-get update -y
!sudo apt-get install python3.9 python3-pip

# Install inspect4py
!python3.9 -m pip install -U inspect4py==0.0.6

In [ ]:
!inspect4py --version

inspect4py, version 0.0.6


### Download test repositories and run `inspect4py` on them

In [ ]:
# Repo-topic pairs are generated from 
# https://github.com/RepoAnalysis/RepoSim/blob/main/data/repo_topic.py
REPOS = {
    'keon/algorithms': 'Algorithms',
    'prabhupant/python-ds': 'Algorithms',
    'grantjenks/python-sortedcontainers': 'Algorithms',
    'TheAlgorithms/Python': 'Algorithms',
    'beetbox/audioread': 'Audio',
    'worldveil/dejavu': 'Audio',
    'keunwoochoi/kapre': 'Audio',
    'librosa/librosa': 'Audio',
    'sergree/matchering': 'Audio',
    'tyiannak/pyAudioAnalysis': 'Audio',
    'jiaaro/pydub': 'Audio',
    'Parisson/TimeSide': 'Audio',
    'lepture/authlib': 'OAuth',
    'pennersr/django-allauth': 'OAuth',
    'evonove/django-oauth-toolkit': 'OAuth',
    'idan/oauthlib': 'OAuth',
    'joestump/python-oauth2': 'OAuth',
    'omab/python-social-auth': 'OAuth',
    'paramiko/paramiko': 'Cryptography',
    'pyca/pynacl': 'Cryptography',
    'jindaxiang/akshare': 'Downloader',
    's3tools/s3cmd': 'Downloader',
    'bloomreach/s4cmd': 'Downloader',
    'euske/pdfminer': 'PDF',
    'mstamy2/PyPDF2': 'PDF',
    'lepture/mistune': 'Markdown',
    'waylan/Python-Markdown': 'Markdown'
}

In [ ]:
!mkdir -p /content/output
%cd /content/
for repo in REPOS:
    !mkdir -p {repo} && git clone {f"https://github.com/{repo}.git"} {repo}
    !inspect4py -i /content/{repo} -o /content/output/{repo} -sc -rm

### Extract docstrings and functions from repositories.

In [ ]:
import json

def funcs_to_lists(funcs, func_codes, docs):
    for func_name, func_info in funcs.items():
        if func_info.get("source_code") is not None:
            func_codes.append(func_info["source_code"])
        if func_info.get("doc") is None:
            continue
        for key in ["full", "long_description", "short_description"]:
            if func_info["doc"].get(key) is not None:
                docs.append(f"{func_name} {func_info['doc'].get(key)}")
                break


def file_to_lists(filename):
    func_codes = []
    docs = []
    with open(filename, "r") as f:
        dic = json.load(f)
    dic.pop("readme_files", None)
    for dir_name, files in dic.items():
        for file in files:
            if file.get("functions") is not None:
                funcs_to_lists(file["functions"], func_codes, docs)
            if file.get("classes") is not None:
                for class_name, class_info in file["classes"].items():
                    if class_info.get("methods") is not None:
                        funcs_to_lists(class_info["methods"], func_codes, docs)

    return func_codes, docs


In [ ]:
repo_info = {}
for repo, topic in REPOS.items():
    repo_info[repo] = {}

    function_list, docstring_list = file_to_lists(f"/content/output/{repo}/directory_info.json")
    repo_info[repo]["docs"] = docstring_list
    repo_info[repo]["funcs"] = function_list
    repo_info[repo]["topic"] = topic

### Download UniXCoder, fine-tuned model and install requirements

In [ ]:
%cd /content/
%pip install -U sentence-transformers
!wget https://raw.githubusercontent.com/microsoft/CodeBERT/master/UniXcoder/unixcoder.py

### Generate embeddings for all repositories

In [ ]:
import torch
from unixcoder import UniXcoder
from transformers import RobertaModel
from sentence_transformers import SentenceTransformer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
doc_model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2", device=device)
code_model = UniXcoder("Lazyhope/unixcoder-nine-advtest")
code_model.to(device)

def get_code_embeddings(code):
    tokens_ids = code_model.tokenize([code], max_length=512, mode="<encoder-only>")
    source_ids = torch.tensor(tokens_ids).to(device)
    _, embeddings = code_model(source_ids)

    return embeddings

def get_repo_embeddings(lst, input_type):
    if not lst:
        return None
    with torch.no_grad():
        if input_type == "code":
            embeddings_list = torch.concat([get_code_embeddings(code) for code in lst])
        elif input_type == "doc":
            embeddings_list = doc_model.encode(lst, convert_to_tensor=True)
    
        mean_embeddings = torch.mean(embeddings_list, axis=0)

    return mean_embeddings

In [ ]:
from tqdm import tqdm


# Generate code and docstring embeddings for all repositories
for repo_name, repo_dict in tqdm(repo_info.items()):
    print(f" - Generating embeddings for {repo_name} - ")
    if repo_dict.get("code_embeddings") is None:
        repo_dict["code_embeddings"] = get_repo_embeddings(repo_dict["funcs"], input_type="code")
    if repo_dict.get("doc_embeddings") is None:
        repo_dict["doc_embeddings"] = get_repo_embeddings(repo_dict["docs"], input_type="doc")

  0%|          | 0/27 [00:00<?, ?it/s]

 - Generating embeddings for keon/algorithms - 


  4%|▎         | 1/27 [00:27<11:59, 27.69s/it]

 - Generating embeddings for prabhupant/python-ds - 


  7%|▋         | 2/27 [00:33<06:17, 15.11s/it]

...

 93%|█████████▎| 25/27 [04:52<00:15,  7.84s/it]

 - Generating embeddings for lepture/mistune - 


 96%|█████████▋| 26/27 [04:55<00:06,  6.64s/it]

 - Generating embeddings for waylan/Python-Markdown - 


100%|██████████| 27/27 [05:11<00:00, 11.54s/it]


### Evaluations & Results

In [ ]:
from torch.nn import CosineSimilarity
from itertools import combinations

cossim = CosineSimilarity(dim=0, eps=1e-8)
res = []
for repo1, repo2 in combinations(REPOS, 2):
    code_embeddings1 = repo_info[repo1]["code_embeddings"]
    code_embeddings2 = repo_info[repo2]["code_embeddings"]
    if code_embeddings1 is None or code_embeddings2 is None:
        code_similarity = None
    else:
        code_similarity = cossim(code_embeddings1, code_embeddings2).cpu().detach().numpy().item()

    doc_embeddings1 = repo_info[repo1]["doc_embeddings"]
    doc_embeddings2 = repo_info[repo2]["doc_embeddings"]
    if doc_embeddings1 is None or doc_embeddings2 is None:
        doc_similarity = None
    else:
        doc_similarity = cossim(doc_embeddings1, doc_embeddings2).cpu().detach().numpy().item()

    topic1 = repo_info[repo1]["topic"]
    topic2 = repo_info[repo2]["topic"]

    res.append((repo1, repo2, topic1, topic2, code_similarity, doc_similarity))

In [ ]:
import pandas as pd

df = pd.DataFrame(res, columns=["repo1", "repo2", "topic1", "topic2", "code_sim", "doc_sim"])

# NaN values due to missing code/docstring in the repo will be skipped
df["avg_sim"] = df[["code_sim", "doc_sim"]].mean(axis=1, skipna=True)
df

,repo1,repo2,topic1,topic2,code_sim,doc_sim,avg_sim
0,keon/algorithms,prabhupant/python-ds,Algorithms,Algorithms,0.778547,0.879596,0.829071
1,keon/algorithms,grantjenks/python-sortedcontainers,Algorithms,Algorithms,0.727102,0.867281,0.797192
2,keon/algorithms,TheAlgorithms/Python,Algorithms,Algorithms,0.895993,0.900286,0.898139
3,keon/algorithms,beetbox/audioread,Algorithms,Audio,0.008753,0.471483,0.240118
4,keon/algorithms,worldveil/dejavu,Algorithms,Audio,0.176522,0.522964,0.349743
...,...,...,...,...,...,...,...
346,euske/pdfminer,lepture/mistune,PDF,Markdown,0.294981,0.450331,0.372656
347,euske/pdfminer,waylan/Python-Markdown,PDF,Markdown,0.305095,0.414341,0.359718
348,mstamy2/PyPDF2,lepture/mistune,PDF,Markdown,0.333942,0.780140,0.557041
349,mstamy2/PyPDF2,waylan/Python-Markdown,PDF,Markdown,0.532385,0.814950,0.673668


In [ ]:
# Sort the table based on code semantic similarity
df.sort_values("code_sim", ascending=False).reset_index(drop=True)

,repo1,repo2,topic1,topic2,code_sim,doc_sim,avg_sim
0,lepture/authlib,idan/oauthlib,OAuth,OAuth,0.936125,0.959024,0.947574
1,evonove/django-oauth-toolkit,idan/oauthlib,OAuth,OAuth,0.922530,0.932178,0.927354
2,keon/algorithms,TheAlgorithms/Python,Algorithms,Algorithms,0.895993,0.900286,0.898139
3,lepture/authlib,evonove/django-oauth-toolkit,OAuth,OAuth,0.891102,0.878009,0.884556
4,idan/oauthlib,joestump/python-oauth2,OAuth,OAuth,0.879883,0.919387,0.899635
...,...,...,...,...,...,...,...
346,sergree/matchering,pennersr/django-allauth,Audio,OAuth,-0.127733,NaN,-0.127733
347,tyiannak/pyAudioAnalysis,pennersr/django-allauth,Audio,OAuth,-0.127950,0.253006,0.062528
348,jindaxiang/akshare,s3tools/s3cmd,Downloader,Downloader,-0.128894,0.235784,0.053445
349,librosa/librosa,pennersr/django-allauth,Audio,OAuth,-0.134879,0.254131,0.059626


In [ ]:
# Sort the table based on docstring semantic similarity
df.sort_values("doc_sim", ascending=False).reset_index(drop=True)

,repo1,repo2,topic1,topic2,code_sim,doc_sim,avg_sim
0,lepture/authlib,idan/oauthlib,OAuth,OAuth,0.936125,0.959024,0.947574
1,evonove/django-oauth-toolkit,idan/oauthlib,OAuth,OAuth,0.922530,0.932178,0.927354
2,idan/oauthlib,joestump/python-oauth2,OAuth,OAuth,0.879883,0.919387,0.899635
3,keon/algorithms,TheAlgorithms/Python,Algorithms,Algorithms,0.895993,0.900286,0.898139
4,lepture/authlib,joestump/python-oauth2,OAuth,OAuth,0.834853,0.899069,0.866961
...,...,...,...,...,...,...,...
346,sergree/matchering,bloomreach/s4cmd,Audio,Downloader,0.003936,NaN,0.003936
347,sergree/matchering,euske/pdfminer,Audio,PDF,0.147579,NaN,0.147579
348,sergree/matchering,mstamy2/PyPDF2,Audio,PDF,0.180949,NaN,0.180949
349,sergree/matchering,lepture/mistune,Audio,Markdown,0.022897,NaN,0.022897


In [ ]:
# Sort the table based on average similarity and save it
df = df.sort_values("avg_sim", ascending=False).reset_index(drop=True)
df.to_csv("eval_res.csv", index=False)

df

,repo1,repo2,topic1,topic2,code_sim,doc_sim,avg_sim
0,lepture/authlib,idan/oauthlib,OAuth,OAuth,0.936125,0.959024,0.947574
1,evonove/django-oauth-toolkit,idan/oauthlib,OAuth,OAuth,0.922530,0.932178,0.927354
2,idan/oauthlib,joestump/python-oauth2,OAuth,OAuth,0.879883,0.919387,0.899635
3,keon/algorithms,TheAlgorithms/Python,Algorithms,Algorithms,0.895993,0.900286,0.898139
4,lepture/authlib,evonove/django-oauth-toolkit,OAuth,OAuth,0.891102,0.878009,0.884556
...,...,...,...,...,...,...,...
346,sergree/matchering,lepture/authlib,Audio,OAuth,-0.036973,NaN,-0.036973
347,sergree/matchering,omab/python-social-auth,Audio,OAuth,-0.047977,NaN,-0.047977
348,sergree/matchering,evonove/django-oauth-toolkit,Audio,OAuth,-0.061241,NaN,-0.061241
349,sergree/matchering,s3tools/s3cmd,Audio,Downloader,-0.061914,NaN,-0.061914
